In [23]:
import requests
import csv
import pandas as pd
from io import StringIO

In [3]:
#URLs
BASE_URL = "https://www.nseindia.com"
COOKIE_URL = f"{BASE_URL}/get-quotes/equity?symbol=RELIANCE"
API_URL = "https://www.nseindia.com/api/quote-equity?symbol={symbol}"
SYMBOL_LIST_URL = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"

# Headers for requests
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": BASE_URL
}

In [18]:
import ocifs
import os

oci_config = os.environ.get("OCI_CONFIG")
oci_key_content = os.environ.get("OCI_KEY")
oci_bucket = os.environ.get("OCI_BUCKET")

if oci_config is None or oci_key_content is None or oci_bucket is None:
    raise KeyError("Missing OCI config")

OCI_PRIVATE_KEY_PATH = "./key.pem"
with open(OCI_PRIVATE_KEY_PATH, "w") as key_file:
    key_file.write(oci_key_content)
    OCI_PRIVATE_KEY_PATH = key_file.name  # Full URL

OCI_CONFIG_PATH = "./config"
with open(OCI_CONFIG_PATH, "w") as config_file:
    oci_config += f'\nkey_file={OCI_PRIVATE_KEY_PATH}'
    config_file.write(oci_config)
    OCI_CONFIG_PATH = config_file.name

storage_options = {"config": OCI_CONFIG_PATH}
print("OCI FS Configured")

KeyError: 'Missing OCI config'

In [22]:
def fetch_symbol_list():
    """Fetch the list of symbols from NSE's EQUITY_L.csv"""
    response = requests.get(SYMBOL_LIST_URL, headers=HEADERS)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))
    return df["SYMBOL"].dropna().tolist()  # Extract only the SYMBOL column

In [29]:
def fetch_nse_industry_info(symbols):
    """Fetch industry info for symbols and save to a CSV file using pandas"""
    with requests.Session() as session:
        # Load initial cookie
        session.get(COOKIE_URL, headers=HEADERS)
        industry_data = []  # List to store data

        for symbol in symbols:
            try:
                url = API_URL.format(symbol=symbol)
                response = session.get(url, headers=HEADERS)
                response.raise_for_status()
                data = response.json()

                # Extract industry information
                industry_info = data.get("industryInfo", {})
                macro = industry_info.get("macro", "N/A")
                sector = industry_info.get("sector", "N/A")
                industry = industry_info.get("industry", "N/A")
                basic_industry = industry_info.get("basicIndustry", "N/A")

                # Append to list
                industry_data.append([symbol, macro, sector, industry, basic_industry])
                print(f"Data for {symbol} saved.")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching data for {symbol}: {e}")

        # Convert to DataFrame and save to CSV
        return pd.DataFrame(industry_data, columns=["Symbol", "Macro", "Sector", "Industry", "Basic Industry"])


In [30]:
symbols_list = fetch_symbol_list()[0:10]

In [31]:
df = fetch_nse_industry_info(symbols_list)

Data for 20MICRONS saved.
Data for 21STCENMGM saved.
Data for 360ONE saved.
Data for 3IINFOLTD saved.
Data for 3MINDIA saved.
Data for 3PLAND saved.
Data for 5PAISA saved.
Data for 63MOONS saved.
Data for A2ZINFRA saved.
Data for AAATECH saved.


In [32]:
df.to_csv(f'oci://{oci_bucket}/nse_industry_symbols.csv', index=False)
print(f"Data saved")

Data saved
